# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (nonlinear problem)

In this tutorial we solve the problem

$$\begin{align*}
&\min_{u} \int_\Omega \left\{ (1 + u^2)\ |\nabla u|^2 - u \right\} dx,\\
&\text{s.t. } u = g\text{ on }\Gamma = \partial \Omega
\end{align*}$$
where $\Omega$ is the unit ball in 2D.

The optimality conditions result in the following nonlinear problem

$$\begin{align*}
&\int_\Omega (1+u^2)\ \nabla u \cdot \nabla v dx + \int_\Omega u \ |\nabla u|^2 v dx = \int_\Omega v dx\\
&\text{s.t. } u = g\text{ on }\Gamma = \partial \Omega
\end{align*}$$


We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega (1+u^2)\ \nabla u \cdot \nabla v dx + \int_\Omega u \ |\nabla u|^2 v dx = \int_\Omega v dx, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega (1+u^2)\ \nabla u \cdot \nabla v dx + \int_\Omega u \ |\nabla u|^2 v dx + \int_\Gamma \lambda v = \int_\Omega v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example is a prototypical case of problems containing subdomain/boundary restricted variables (the Lagrange multiplier, in this case).

In [ ]:
from numpy import arange, isclose, sin, sqrt, where
from petsc4py import PETSc
from ufl import derivative, grad, inner, Measure, replace
from dolfinx import *
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import assemble_matrix, assemble_scalar, assemble_vector, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot
from multiphenics import *
from multiphenics.fem import DirichletBCLegacy

### Mesh

In [ ]:
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_size_t(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_size_t(mesh)
facets_Gamma = where(boundaries.values == 1)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Weak imposition of Dirichlet BCs

In [ ]:
# Define a block function space
V = FunctionSpace(mesh, ("Lagrange", 2))
dofs_V = arange(0, V.dofmap.index_map.block_size*(V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts))
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
W = BlockFunctionSpace([V, V], restrict=[dofs_V, dofs_V_Gamma])

In [ ]:
# Define trial and test functions, as well as solution
dul = BlockTrialFunction(W)
(du, dl) = block_split(dul)
ul = BlockFunction(W)
(u, l) = block_split(ul)
vm = BlockTestFunction(W)
(v, m) = block_split(vm)

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: sin(3*x[0] + 1)*sin(3*x[1] + 1))
F = [inner((1+u**2)*grad(u), grad(v))*dx + u*v*inner(grad(u), grad(u))*dx + l*v*ds - v*dx,
     u*m*ds - g*m*ds]
F = BlockForm1(F, [W])
J = block_derivative(F, ul, dul)

In [ ]:
# Solve
def set_solver_parameters(solver):
    solver.max_it = 20

problem = BlockNonlinearProblem(F, ul, None, J)
solver = BlockNewtonSolver(mesh.mpi_comm())
set_solver_parameters(solver)
solver.solve(problem, ul.block_vector)

In [ ]:
plot(ul[0])

### Strong imposition of Dirichlet BCs

In [ ]:
# Class for interfacing with the Newton solver
class LagrangeMultipliersNonlinearProblem(NonlinearProblem):
    def __init__(self, F, up, bc, J):
        NonlinearProblem.__init__(self)
        self._F = F
        self._up = up
        self._bc = bc
        self._J = J
        self._F_vec = None
        self._J_mat = None

    def form(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    def F(self, _):
        if self._F_vec is None:
            self._F_vec = assemble_vector(self._F)
        else:
            with self._F_vec.localForm() as f_local:
                f_local.set(0.0)
            assemble_vector(self._F_vec, self._F)
        self._F_vec.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        DirichletBCLegacy.apply(self._bc, self._F_vec, self._up.vector)
        return self._F_vec

    def J(self, _):
        if self._J_mat is None:
            self._J_mat = assemble_matrix(self._J)
        else:
            self._J_mat.zeroEntries()
            assemble_matrix(self._J_mat, self._J)
        self._J_mat.assemble()
        DirichletBCLegacy.apply(self._bc, self._J_mat, 1.0)
        return self._J_mat

In [ ]:
# Define problem block forms
u_ex = Function(V)
F_ex = replace(F[0], {u: u_ex, l: 0})
J_ex = derivative(F_ex, u_ex, du)

In [ ]:
# Define Dirichlet BC object on Gamma
bc_ex = [DirichletBC(g, dofs_V_Gamma)]

In [ ]:
# Solve
problem_ex = LagrangeMultipliersNonlinearProblem(F_ex, u_ex, bc_ex, J_ex)
solver_ex = NewtonSolver(mesh.mpi_comm())
set_solver_parameters(solver_ex)
solver_ex.solve(problem_ex, u_ex.vector)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex), grad(u_ex))*dx)))
err_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex - ul[0]), grad(u_ex - ul[0]))*dx)))
print("Relative error is equal to", err_norm/u_ex_norm)
assert isclose(err_norm/u_ex_norm, 0., atol=1.e-9)